In [2]:
import pandas as pd

data=pd.read_csv('./weibo_senti_100k.csv')
data=data.dropna()
data

,label,review
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你]
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...
2,1,姑娘都羡慕你呢…还有招财猫高兴……//@爱在蔓延-JC:[哈哈]小学徒一枚，等着明天见您呢/...
3,1,美~~~~~[爱你]
4,1,梦想有多大，舞台就有多大![鼓掌]
...,...,...
119983,0,一公里不到，县医院那个天桥下右拐200米就到了！//@谢礼恒: 我靠。这个太霸道了！离224...
119984,0,今天真冷啊，难道又要穿棉袄了[晕]？今年的春天真的是百变莫测啊[抓狂]
119985,0,最近几天就没停止过！！！[伤心]
119986,0,//@毒药女流氓:[怒] 很惨!


In [3]:
import jieba
data['data_cut'] = data['review'].apply(lambda x: list(jieba.cut(x))) #内嵌自定义函数来分词
data.head()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\YaliZhu\AppData\Local\Temp\jieba.cache
Loading model cost 0.784 seconds.
Prefix dict has been built successfully.


,label,review,data_cut
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你],"[﻿, 更博, 了, ，, 爆照, 了, ，, 帅, 的, 呀, ，, 就是, 越来越, 爱..."
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...,"[@, 张晓鹏, jonathan, , 土耳其, 的, 事要, 认真对待, [, 哈哈,..."
2,1,姑娘都羡慕你呢…还有招财猫高兴……//@爱在蔓延-JC:[哈哈]小学徒一枚，等着明天见您呢/...,"[姑娘, 都, 羡慕, 你, 呢, …, 还有, 招财猫, 高兴, …, …, /, /, ..."
3,1,美~~~~~[爱你],"[美, ~, ~, ~, ~, ~, [, 爱, 你, ]]"
4,1,梦想有多大，舞台就有多大![鼓掌],"[梦想, 有, 多, 大, ，, 舞台, 就, 有, 多, 大, !, [, 鼓掌, ]]"


In [4]:
# 去停用词
# 读取停用词
with open('stopword.txt','r',encoding = 'utf-8') as f: #读取停用词txt 文档
    stop = f.readlines()
# 对停用词处理
import re
stop = [re.sub(' |\n|\ufeff','',r) for r in stop] #替换停用词表的空格等
# 去除停用词
#把分词之后的文本根据停用词表去掉停用词
data['data_after'] = [[i for i in s if i not in stop] for s in data['data_cut']]
data.head()

,label,review,data_cut,data_after
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你],"[﻿, 更博, 了, ，, 爆照, 了, ，, 帅, 的, 呀, ，, 就是, 越来越, 爱...","[﻿, 更博, 爆照, 帅, 越来越, 爱, 生快, 傻, 缺, 爱, 爱, 爱]"
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...,"[@, 张晓鹏, jonathan, , 土耳其, 的, 事要, 认真对待, [, 哈哈,...","[张晓鹏, jonathan, , 土耳其, 事要, 认真对待, 开除, 丁丁, 世界, ..."
2,1,姑娘都羡慕你呢…还有招财猫高兴……//@爱在蔓延-JC:[哈哈]小学徒一枚，等着明天见您呢/...,"[姑娘, 都, 羡慕, 你, 呢, …, 还有, 招财猫, 高兴, …, …, /, /, ...","[姑娘, 羡慕, 招财猫, 爱, 蔓延, JC, 学徒, 一枚, 明天, 李欣芸, Shar..."
3,1,美~~~~~[爱你],"[美, ~, ~, ~, ~, ~, [, 爱, 你, ]]","[美, 爱]"
4,1,梦想有多大，舞台就有多大![鼓掌],"[梦想, 有, 多, 大, ，, 舞台, 就, 有, 多, 大, !, [, 鼓掌, ]]","[梦想, 舞台, 鼓掌]"


In [5]:
# 构建词向量矩阵
w = []
for i in data['data_after']:
    w.extend(i) #将所有词语整合在一起
num_data = pd.DataFrame(pd.Series(w).value_counts()) # 计算出所有单词的个数
num_data['id'] = list(range(1,len(num_data)+1)) #把这些数据增加序号
# 转化成数字
a = lambda x:list(num_data['id'][x]) #以序号为序定义实现函数
data['vec'] = data['data_after'].apply(a) #apply（）方法实现
data.head()

,label,review,data_cut,data_after,vec
0,1,﻿更博了，爆照了，帅的呀，就是越来越爱你！生快傻缺[爱你][爱你][爱你],"[﻿, 更博, 了, ，, 爆照, 了, ，, 帅, 的, 呀, ，, 就是, 越来越, 爱...","[﻿, 更博, 爆照, 帅, 越来越, 爱, 生快, 傻, 缺, 爱, 爱, 爱]","[83982, 53281, 56556, 481, 312, 4, 2270, 451, ..."
1,1,@张晓鹏jonathan 土耳其的事要认真对待[哈哈]，否则直接开除。@丁丁看世界 很是细心...,"[@, 张晓鹏, jonathan, , 土耳其, 的, 事要, 认真对待, [, 哈哈,...","[张晓鹏, jonathan, , 土耳其, 事要, 认真对待, 开除, 丁丁, 世界, ...","[8590, 9062, 1, 4948, 29179, 49557, 13038, 217..."
2,1,姑娘都羡慕你呢…还有招财猫高兴……//@爱在蔓延-JC:[哈哈]小学徒一枚，等着明天见您呢/...,"[姑娘, 都, 羡慕, 你, 呢, …, 还有, 招财猫, 高兴, …, …, /, /, ...","[姑娘, 羡慕, 招财猫, 爱, 蔓延, JC, 学徒, 一枚, 明天, 李欣芸, Shar...","[275, 484, 19532, 4, 3078, 6659, 9211, 1979, 7..."
3,1,美~~~~~[爱你],"[美, ~, ~, ~, ~, ~, [, 爱, 你, ]]","[美, 爱]","[126, 4]"
4,1,梦想有多大，舞台就有多大![鼓掌],"[梦想, 有, 多, 大, ，, 舞台, 就, 有, 多, 大, !, [, 鼓掌, ]]","[梦想, 舞台, 鼓掌]","[326, 1466, 6]"


In [ ]:
# 构建词云
#加载词云库
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# ## 词频统计
# 重组词组
num_words = [''.join(i) for i in data['data_after']] #把所有词提取出来
num_words = ''.join(num_words) # 把所有词拼接成一个字符串
num_words= re.sub(' ','',num_words) #去除空格
# 计算全部词频
num = pd.Series(jieba.lcut(num_words)).value_counts()
# 用wordcloud 画图
wc_pic =WordCloud(background_color='white',font_path=r'C:\Windows\Fonts\simhei.ttf').fit_words(num)
plt.figure(figsize=(15,15)) #图片大小定义
plt.imshow(wc_pic)#输出图片
plt.axis('off')#不显示坐标轴
plt.show()

In [12]:
# ## 数据集划分
import torch
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import numpy as np
# maxlen = 100 #句子长度
data_vec = list(data['vec'].values)
data_label = torch.tensor(list(data['label'].values),dtype=torch.float32)
vec_data = pad_sequence([torch.from_numpy(np.array(x)) for x in data_vec],batch_first=True)#把文本数据都统一长度
x_train,x_test,y_train,y_test = train_test_split(vec_data,data_label,test_size = 0.2,random_state= 124) #分割训练集--2-8 原则

SVM

In [ ]:
from sklearn.svm import SVC
clf = SVC(C=1, kernel = 'linear',max_iter=2000) # 用线性分类器
clf.fit(x,y) # #模型训练
# 调用报告
from sklearn.metrics import classification_report
test_pre = clf.predict(xt) # 模型预测
report = classification_report(yt,test_pre) #预测结果
print(report)

LSTM

In [98]:
from torch.nn import Embedding,LSTM,Linear,Sigmoid,Sequential,Flatten,Module
import torch.nn.functional as F
class Models(Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(Models, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = Embedding(vocab_size, embedding_dim)
        self.lstm =LSTM(input_size=embedding_dim,hidden_size=hidden_dim,num_layers=1,batch_first=True)
        self.linear2 = Linear(hidden_dim, tagset_size)

        self.hidden = self.init_hidden()

    def init_hidden(self):
        # 一开始并没有隐藏状态所以我们要先初始化一个
        # 关于维度为什么这么设计请参考Pytoch相关文档
        # 各个维度的含义是 (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        out = self.embeddings(sentence)
        # out = self.linear1(out)   #1,123seq_len,128hidden
        lstm_out, self.hidden = self.lstm(out, self.hidden)
        tag_space = self.linear2(lstm_out[...,-1,:])
        tag_scores = F.sigmoid(tag_space)
        return tag_scores


In [99]:
model=Models(embedding_dim=256, hidden_dim=128, vocab_size=201995, tagset_size=1)
model

Models(
  (embeddings): Embedding(201995, 256)
  (lstm): LSTM(256, 128, batch_first=True)
  (linear2): Linear(in_features=128, out_features=1, bias=True)
)

In [62]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
loss=torch.nn.BCELoss().cuda()

In [102]:
from tqdm import tqdm
# 模型训练
epochs=10
for epoch in range(epochs):
    model.train()
    for x,y in tqdm(zip(x_train,y_train)):
        optimizer.zero_grad()
        x=x.reshape(1,x.shape[0])   #batch size=1
        print(x.shape)
        y=y.reshape(1,1)
        x=x.to(device)
        y=y.to(device)
        y_pred=model(x)
        loss_=loss(y_pred,y)
        loss_.backward()
        optimizer.step()
        
    model.eval()
    with torch.no_grad():
        y_pred=model(x_test)
        y_pred=y_pred.squeeze()
        y_pred=y_pred>0.5
        y_pred=y_pred.cpu().numpy()
        y_test=y_test.cpu().numpy()
        print(classification_report(y_test,y_pred))

0it [00:00, ?it/s]

torch.Size([1, 123])


0it [00:00, ?it/s]


RuntimeError: Trying to backward through the graph a second time (or directly access saved variables after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved variables after calling backward.

In [ ]:
#模型验证
loss,accuracy=model.(x_test,y_test,batch_size=12) # 测试集评估
print('Test loss:',loss)
print('Test accuracy:', accuracy)